In [2]:
!pip install info-gain

In [3]:
import os
import pandas as pd
import numpy as np

from info_gain import info_gain

In [7]:
import json
kaggle_entity = json.load(open('/content/kaggle.json'))
KAGGLE_USERNAME = kaggle_entity['username']
KAGGLE_API_KEY = kaggle_entity['key']

os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME
os.environ['KAGGLE_KEY'] = KAGGLE_API_KEY

In [10]:
!kaggle datasets download merishnasuwal/breast-cancer-prediction-dataset

  0% 0.00/7.92k [00:00<?, ?B/s]
100% 7.92k/7.92k [00:00<00:00, 3.64MB/s]


In [20]:
# if 'Breast_cancer_data.csv' not in os.listdir('/content/'):
#   !unzip -q /content/breast-cancer-prediction-dataset.zip

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data

--2021-09-12 07:53:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51867 (51K) [application/x-httpd-php]
Saving to: ‘car.data’

car.data            100%[===================>]  50.65K  --.-KB/s    in 0.1s    

2021-09-12 07:53:10 (391 KB/s) - ‘car.data’ saved [51867/51867]



In [140]:
df = pd.read_csv('/content/car.data', header=None)
df = df.loc[(df[6] == 'unacc') | (df[6] == 'acc')]

'''
attribute : 
buying: vhigh, high, med, low.
maint: vhigh, high, med, low.
doors: 2, 3, 4, 5more.
persons: 2, 4, more.
lug_boot: small, med, big.
safety: low, med, high.
'''

df.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'values']
df.head()
# df['values'].unique().tolist()

'unacc'

In [46]:
info_gain.info_gain(df['values'].values, df['buying'].values)

0.00683644420760432

In [29]:
df.describe()

,0,1,2,3,4,5,6
count,1594,1594,1594,1594,1594,1594,1594
unique,4,4,4,3,3,3,2
top,vhigh,vhigh,2,2,small,low,unacc
freq,432,432,407,576,555,576,1210


In [78]:
features = df.columns[:-1].tolist()
features

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']

In [182]:
def base_case(df_, features_, y):

  un = df_[y].unique()
  if len(un) == 1:
    return un[0], True
  
  if len(features_) == 0:
    return df_[y].max(), True

  return '', False

In [199]:
def generate_tree(feature, features_, value, df_, y):
  target, base = base_case(df_, features_, y)
  print(base)
  # print(target)
  if base:
    print(y, target) 
  else:
    # cari feature dengan information_gain tertinggi
    max = -100000
    max_f = ''
    for f in features_:
      gain = info_gain.info_gain_ratio(df_[y].values, df_[f].values)
      if max < gain:
        max_f, max = f, gain

    # print(max_f)
    # print(features_copy)
    list_values = df_[max_f].unique()
    features_copy = features_.copy()
    features_copy.remove(max_f)
    for v in list_values:
      df_copy = df_.copy()
      print('this -> ' + max_f)
      df_copy = df_copy.loc[df_copy[max_f] == v]
      print(df_copy)
      generate_tree(max_f, features_copy, v, df_copy, y)

In [170]:
df_0 = pd.DataFrame({
    'Fever' : [False, True, True, True, True, False, True, True, False, True, False, False, False, True],
    'Cough' : [False, True, True, False, True, True, False, False, True, True, True, True, True, True],
    'Breathing issues' : [False, True, False, True, True, False, True, True, True, False, False, True, True, False],
    'Infected' : [False, True, False, True, True, False, False, True, True, True, True, False, True, False]
})

df_0

,Fever,Cough,Breathing issues,Infected
0,False,False,False,False
1,True,True,True,True
2,True,True,False,False
3,True,False,True,True
4,True,True,True,True
5,False,True,False,False
6,True,False,True,False
7,True,False,True,True
8,False,True,True,True
9,True,True,False,True


In [ ]:
cols = df_0.columns[:-1].tolist()
df_0_copy = df_0.copy()
# df_copy = df.copy()

generate_tree('', cols, '', df_0_copy, 'Infected')